In [5]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.appName("ReadJSON").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/14 15:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType
#"reviewerID": "A8WEXFRWX1ZHH", 
# "asin": "0209688726", 
# "style": {"Color:": " AC"}, 
# "reviewerName": "Goldengate",
# Define the schema
schema = StructType([
    StructField("overall", FloatType(), True),
    StructField("verified", BooleanType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("style", StructType([StructField("Color:", StringType(), True)]), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True)
    
])


json_df = spark.read.schema(schema).json("combined_train_data_chunked_10mb_latest.json")
json_test_df = spark.read.schema(schema).json("combined_test_data_chunked_10mb_latest.json")
json_df.show(5)
json_test_df.show(5)

# 

+-------+--------+-----------+--------------+----------+------+--------------+--------------------+--------------+
|overall|verified| reviewTime|    reviewerID|      asin| style|  reviewerName|          reviewText|unixReviewTime|
+-------+--------+-----------+--------------+----------+------+--------------+--------------------+--------------+
|    5.0|    true| 04 5, 2016|A1274GG1EB2JLJ|0486427706|{NULL}|   barbara ann|The pictures are ...|    1459814400|
|    5.0|    true|02 13, 2016|A30X5EGBYAZQQK|0486427706|{NULL}|      Samantha|I absolutely love...|    1455321600|
|    5.0|    true|12 10, 2015|A3U6UNXLAUY6ZV|0486427706|{NULL}|   CP in Texas|          I love it!|    1449705600|
|    5.0|    true|10 26, 2015|A1SAJF5SNM6WJS|0486427706|{NULL}|   LOIS LABIER|MY HUSBAND LOVED ...|    1445817600|
|    4.0|    true|09 15, 2015| AHJWO3SI0S0OR|0486427706|{NULL}|Saundra Hatley|                cool|    1442275200|
+-------+--------+-----------+--------------+----------+------+--------------+--

In [7]:
# Print the number of entries in the dataframe
print("Number of entries in the dataframe before pre processing: ", json_df.count())
print("Number of entries in the dataframe before pre processing: ", json_test_df.count())

Number of entries in the dataframe before pre processing:  11321389


Number of entries in the dataframe before pre processing:  2832499


In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType
# targetUDF = F.udf(lambda x: 1 if x >= 4.0 else (0 if x == 3.0 else -1), IntegerType())
targetUDF = F.udf(lambda x: 1 if x >= 4.0 else 0, IntegerType())
import re
import nltk
from nltk.corpus import stopwords

In [9]:
reduced_df = json_df.select("overall", "reviewerID", "asin", "reviewText")
reduced_test_df = json_test_df.select("overall", "reviewerID", "asin", "reviewText")



In [10]:
reduced_df.show(5)
reduced_test_df.show(5)

+-------+--------------+----------+--------------------+
|overall|    reviewerID|      asin|          reviewText|
+-------+--------------+----------+--------------------+
|    5.0|A1274GG1EB2JLJ|0486427706|The pictures are ...|
|    5.0|A30X5EGBYAZQQK|0486427706|I absolutely love...|
|    5.0|A3U6UNXLAUY6ZV|0486427706|          I love it!|
|    5.0|A1SAJF5SNM6WJS|0486427706|MY HUSBAND LOVED ...|
|    4.0| AHJWO3SI0S0OR|0486427706|                cool|
+-------+--------------+----------+--------------------+
only showing top 5 rows

+-------+--------------+----------+--------------------+
|overall|    reviewerID|      asin|          reviewText|
+-------+--------------+----------+--------------------+
|    5.0|A2LSCFZM2FBZK7|0486427706|The stained glass...|
|    5.0|A3IXP5VS847GE5|0486427706|My 11 y.o. loved ...|
|    5.0|A2HK5AVQW6AUQ5|0486427706|             love it|
|    5.0|A18MVTKTTE8OS8|0486448789|Sometimes you nee...|
|    5.0|A2C2TLRMMMLAJV|0486448789|These little book...|
+-----

In [11]:
unique_df = reduced_df.dropDuplicates(["reviewerID", "asin"])
print("Number of training entries in the dataframe after removing duplicates: ", unique_df.count())

unique_test_df = reduced_test_df.dropDuplicates(["reviewerID", "asin"])
print("Number of testing entries in the dataframe after removing duplicates: ", unique_test_df.count())

Number of training entries in the dataframe after removing duplicates:  10936206


Number of testing entries in the dataframe after removing duplicates:  2806599


In [12]:
def preProcess(text):
    # Should return a list of tokens
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)    
    text = text.lower()
    tokens = word_tokenize(text)    
    return tokens

In [13]:
df_sentiment = unique_df.withColumn("sentiment", targetUDF(unique_df["overall"]))
df_test_sentiment = unique_test_df.withColumn("sentiment", targetUDF(unique_test_df["overall"]))

In [14]:
df_sentiment.show(5)
df_test_sentiment.show(5)

/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=67445) is multi-threaded, use of fork() may lead to deadlocks in the child.


+-------+--------------------+----------+--------------------+---------+
|overall|          reviewerID|      asin|          reviewText|sentiment|
+-------+--------------------+----------+--------------------+---------+
|    5.0|A0015332H21AK8WZ0ZCS|B005G030TC|These collars are...|        1|
|    4.0| A0020356UF96ZV361ST|B00ZI5OVFM|It is a love stor...|        1|
|    2.0| A0020356UF96ZV361ST|B015X7KEDM|This book is not ...|        0|
|    5.0|A0024936S1WI02OHH9DP|B016AG5DR2|Looks great fits ...|        1|
|    5.0|A0034986DWR7WEDQN0GV|B001VJZO2S|           excellent|        1|
+-------+--------------------+----------+--------------------+---------+
only showing top 5 rows



+-------+-------------------+----------+--------------------+---------+
|overall|         reviewerID|      asin|          reviewText|sentiment|
+-------+-------------------+----------+--------------------+---------+
|    3.0|A0020356UF96ZV361ST|B00FDXFFW2|I guess you can s...|        0|
|    4.0|A0020356UF96ZV361ST|B00H6VZ0SS|This girl has bee...|        1|
|    4.0|A0020356UF96ZV361ST|B00XQOGWV8|Mario is a sorry ...|        1|
|    4.0|A0020356UF96ZV361ST|B014HD23EQ|This father had h...|        1|
|    4.0|A0020356UF96ZV361ST|B018RSH2FW|This guy is and h...|        1|
+-------+-------------------+----------+--------------------+---------+
only showing top 5 rows



In [15]:
from pyspark.ml.feature import Tokenizer

# use PySparks build in tokenizer to tokenize tweets
tokenizer = Tokenizer(inputCol  = "reviewText",
                      outputCol = "token")
# Remove the rows with missing values and tokenize
df_train_tokenized = tokenizer.transform(df_sentiment.filter(unique_df.reviewText.isNotNull()))
df_test_tokenized = tokenizer.transform(df_test_sentiment.filter(unique_test_df.reviewText.isNotNull()))

In [16]:

df_train_tokenized.show(5)
df_test_tokenized.show(5)

+-------+--------------------+----------+--------------------+---------+--------------------+
|overall|          reviewerID|      asin|          reviewText|sentiment|               token|
+-------+--------------------+----------+--------------------+---------+--------------------+
|    5.0|A0015332H21AK8WZ0ZCS|B005G030TC|These collars are...|        1|[these, collars, ...|
|    4.0| A0020356UF96ZV361ST|B00ZI5OVFM|It is a love stor...|        1|[it, is, a, love,...|
|    2.0| A0020356UF96ZV361ST|B015X7KEDM|This book is not ...|        0|[this, book, is, ...|
|    5.0|A0024936S1WI02OHH9DP|B016AG5DR2|Looks great fits ...|        1|[looks, great, fi...|
|    5.0|A0034986DWR7WEDQN0GV|B001VJZO2S|           excellent|        1|         [excellent]|
+-------+--------------------+----------+--------------------+---------+--------------------+
only showing top 5 rows



+-------+-------------------+----------+--------------------+---------+--------------------+
|overall|         reviewerID|      asin|          reviewText|sentiment|               token|
+-------+-------------------+----------+--------------------+---------+--------------------+
|    3.0|A0020356UF96ZV361ST|B00FDXFFW2|I guess you can s...|        0|[i, guess, you, c...|
|    4.0|A0020356UF96ZV361ST|B00H6VZ0SS|This girl has bee...|        1|[this, girl, has,...|
|    4.0|A0020356UF96ZV361ST|B00XQOGWV8|Mario is a sorry ...|        1|[mario, is, a, so...|
|    4.0|A0020356UF96ZV361ST|B014HD23EQ|This father had h...|        1|[this, father, ha...|
|    4.0|A0020356UF96ZV361ST|B018RSH2FW|This guy is and h...|        1|[this, guy, is, a...|
+-------+-------------------+----------+--------------------+---------+--------------------+
only showing top 5 rows



In [17]:
import re

def removeRegex(tokens: list) -> list:
    """
    Removes hashtags, call outs and web addresses from tokens.
    """
    # Use a raw string for regular expressions to avoid escape sequence warnings
    expr = r'(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)|'+\
           r'(https?://[^\s<>"]+|www\.[^\s<>"]+)'
    regex = re.compile(expr)
    cleaned = [t for t in tokens if not regex.search(t) and len(t) > 0]

    return cleaned


In [18]:
removeWEBUDF = F.udf(removeRegex, ArrayType(StringType()))

In [19]:
def normalize(tokens : list) -> list:
    """
    Removes non-english characters and returns lower case versions of words.
    """
    subbed   = [re.sub("[^a-zA-Z]+", "", s).lower() for s in tokens]
    
    filtered = filter(None, subbed)
    
    return list(filtered)


normalizeUDF = F.udf(normalize, ArrayType(StringType()))

In [20]:
# remove hashtags, call outs and web addresses
df4_train = df_train_tokenized.withColumn("tokens_re", removeWEBUDF(df_train_tokenized["token"]))
df4_test = df_test_tokenized.withColumn("tokens_re", removeWEBUDF(df_test_tokenized["token"]))
# remove non english characters
df4_train = df4_train.withColumn("tokens_clean", normalizeUDF(df4_train["tokens_re"]))
df4_test = df4_test.withColumn("tokens_clean", normalizeUDF(df4_test["tokens_re"]))

# rename columns
df5_train = df4_train.drop("token","tokens_re")
df5_test = df4_test.drop("token","tokens_re")
df5_train = df5_train.withColumnRenamed("tokens_clean", "tokens")
df5_test = df5_test.withColumnRenamed("tokens_clean", "tokens")

# remove reviews where the tokens array is empty, i.e. where it was just
# a hashtag, callout, numbers, web adress etc.
df6_train = df5_train.where(F.size(F.col("tokens")) > 0)
df6_test = df5_test.where(F.size(F.col("tokens")) > 0)

In [21]:
df_train_for_model = df6_train.select("reviewText","sentiment")\
        .withColumnRenamed("sentiment", "label")
df_test_for_model = df6_test.select("reviewText","sentiment").withColumnRenamed("sentiment", "label")

In [22]:
from pyspark.sql.functions import rand

# Assuming 'df' is your DataFrame
shuffled_train_df = df_train_for_model.orderBy(rand())
shuffled_test_df = df_test_for_model.orderBy(rand())

# Show the shuffled DataFrame
shuffled_train_df.show(10)
shuffled_test_df.show(10)

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|Positive- works w...|    1|
|Love these.  Work...|    1|
|Bruder's are a go...|    1|
|       I love it!!!!|    1|
|Good amount of po...|    1|
|Design is the pri...|    1|
|Great saw for a g...|    1|
|Good book, You ca...|    1|
|To much sugar, no...|    0|
|Loved the crossov...|    1|
+--------------------+-----+
only showing top 10 rows



+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|This book was way...|    1|
|only one time the...|    1|
|How HEAVENLY are ...|    1|
|Just put it on to...|    1|
|Bright light here...|    1|
|Works great! I ha...|    1|
|I am adding to th...|    1|
|Arrived as expect...|    1|
|This is a fast pa...|    1|
|The claws give it...|    1|
+--------------------+-----+
only showing top 10 rows



In [23]:
# Printing label distribution for training and testing data
print("Training data label distribution")
shuffled_train_df.groupBy("label").count().show()

print("Testing data label distribution")
shuffled_test_df.groupBy("label").count().show()

Training data label distribution


24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:53:47 WARN RowBasedKeyValueBatch: Calling spill() on

+-----+-------+
|label|  count|
+-----+-------+
|    1|9223524|
|    0|1703907|
+-----+-------+

Testing data label distribution


24/04/14 15:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/14 15:55:54 WARN RowBasedKeyValueBatch: Calling spill() on

+-----+-------+
|label|  count|
+-----+-------+
|    1|2368278|
|    0| 435983|
+-----+-------+



In [24]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

# get the name of the metric used
evaluator.getMetricName()

'areaUnderROC'

In [25]:
# import pickle
# with open("model1.pkl", "wb") as f:
#     pickle.dump(model1, f)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

# get the name of the metric used
evaluator.getMetricName()

In [27]:
# create tokens from reviews
tk = Tokenizer(inputCol= "reviewText", outputCol = "tokens")

# create term frequencies for each of the tokens
tf1 = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=1e5)

# create tf-idf for each of the tokens
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=2.0)

# create basic logistic regression model
lr = LogisticRegression(maxIter=20)

In [29]:
from pyspark.ml.feature import StopWordsRemover
sw  = StopWordsRemover(inputCol="tokens", outputCol="filtered")
tf2 = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1e5)